# arXiv Bilingual Reader - TranslateGemma

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jimmyliao/trans-gemma/blob/main/arxiv-reader.ipynb)

**📖 雙語對照閱讀 arXiv 論文，提升英文學術寫作能力**

---

## ✨ 功能特色

- 🎯 **雙語並排**: 原文與翻譯並列，方便對照學習
- 📚 **章節分組**: 依 Abstract、Methods、Results 等結構化組織
- 💾 **互動式 HTML**: 生成可離線閱讀的網頁，支援鍵盤導航（← → 鍵）
- 🔤 **術語表**: 自動提取專業術語及其翻譯
- 🇹🇼 **繁體優化**: 針對台灣繁體中文優化

---

## 🎯 適合對象

- ✅ **研究生**: 閱讀文獻、準備論文寫作
- ✅ **工程師**: 追蹤最新技術、理解前沿研究
- ✅ **英文學習者**: 學習學術英文表達方式

---

## 🚀 支援環境

此 notebook 會**自動偵測**執行環境並調整設定：

- ✅ **Google Colab** (Free T4 GPU) - 推薦新手
- ✅ **GCP Custom Runtime** (T4 GPU) - 進階用戶
- ✅ **本地 Jupyter** (CPU/GPU) - 有 GPU 設備

---

## ⚡ 快速開始

### Google Colab (推薦)
1. 點擊上方 "Open In Colab" 按鈕
2. Runtime → Change runtime type → **T4 GPU** → Save
3. 按順序執行所有 cells（或 Runtime → Run all）

### 預期時間
- **首次執行**: ~10 分鐘（含下載模型 8GB）
- **之後執行**: ~5 分鐘（模型已快取）
- **翻譯速度**: ~3 分鐘/頁

---

## 👤 作者

**Jimmy Liao** ([GitHub](https://github.com/jimmyliao) | [Blog](https://jimmyliao.dev))
- Google AI GDE (Generative AI)
- Microsoft MVP (AI)
- AI Startup CTO

---

**License**: MIT | **Model**: TranslateGemma 4B | **Source**: [GitHub](https://github.com/jimmyliao/trans-gemma)

## Step 0: 環境偵測 & 設定

**自動偵測執行環境並配置**

In [1]:
import sys
import os

# 偵測環境
def detect_environment():
    """Detect runtime environment"""
    # Check if running in Google Colab
    try:
        import google.colab
        return 'colab'
    except ImportError:
        pass
    
    # Check if running on GCP (check for conda py310)
    if os.path.exists('/opt/conda/bin/conda'):
        result = os.popen('/opt/conda/bin/conda env list').read()
        if 'py310' in result:
            return 'gcp'
    
    # Default: local Jupyter
    return 'local'

ENV = detect_environment()
PYTHON_VERSION = f"{sys.version_info.major}.{sys.version_info.minor}"

print("="*80)
print("🔍 Environment Detection")
print("="*80)
print(f"🖥️  Environment: {ENV.upper()}")
print(f"🐍 Python: {PYTHON_VERSION}")
print(f"📂 Working dir: {os.getcwd()}")

# Environment-specific warnings
if ENV == 'gcp' and sys.version_info.minor < 10:
    print("\n⚠️  WARNING: GCP detected but Python < 3.10")
    print("   Please select 'Python 3.10 (trans-gemma)' kernel")
    print("   (Click top-right kernel selector → Python 3.10)")
    raise RuntimeError("Wrong kernel selected")

print("="*80)
print(f"✅ Environment: {ENV.upper()} - Ready!")
print("="*80)

🔍 Environment Detection
🖥️  Environment: GCP
🐍 Python: 3.10
📂 Working dir: /root
✅ Environment: GCP - Ready!


### 🔍 驗證：直接安裝到當前環境（可選）

如果 Step 1 安裝失敗或環境不一致，可以執行以下指令直接安裝到當前 Python 環境：

```python
import sys
print(f"當前 Python: {sys.executable}")
!{sys.executable} -m pip install huggingface_hub transformers accelerate sentencepiece protobuf pymupdf pillow tqdm ipywidgets opencc-python-reimplemented -q
print("\n✅ 安裝完成！")
```

> **注意**: 正常情況下執行 Step 1 即可，此為備用方案。如需使用，請將上方程式碼複製到新 cell 執行。

## Step 1: 安裝套件

**根據環境自動安裝必要套件**

In [6]:
print(f"📦 Installing packages for {ENV.upper()} environment...\n")if ENV == 'colab':    # Google Colab: 使用系統 Python (已經是 3.10+)    !pip install -q huggingface_hub transformers accelerate sentencepiece protobuf pymupdf pillow opencc-python-reimplemented    print("✅ Colab packages installed")    elif ENV == 'gcp':    # GCP: 已經在 py310 環境中，只需安裝基礎套件    !pip install -q huggingface_hub transformers accelerate sentencepiece protobuf pymupdf pillow tqdm ipywidgets opencc-python-reimplemented    print("✅ GCP packages installed")    else:    # Local: 標準安裝    !pip install -q torch torchvision torchaudio huggingface_hub transformers accelerate sentencepiece protobuf pymupdf pillow tqdm opencc-python-reimplemented    print("✅ Local packages installed")print("\n✅ All packages ready!")

📦 Installing packages for GCP environment...

✅ GCP packages installed

✅ All packages ready!


## Step 2: Clone & Install trans-gemma

**取得最新程式碼**

In [14]:
%%bash
# Clone or update repository
if [ -d "trans-gemma" ]; then
    echo "📁 Updating trans-gemma..."
    cd trans-gemma && git pull
else
    echo "📥 Cloning trans-gemma..."
    git clone https://github.com/jimmyliao/trans-gemma.git
fi

# Install
cd trans-gemma
pip install -q -e ".[examples]"

echo "✅ trans-gemma installed"

📁 Updating trans-gemma...


hint: Pulling without specifying how to reconcile divergent branches is
hint: discouraged. You can squelch this message by running one of the following
hint: commands sometime before your next pull:
hint: 
hint:   git config pull.rebase false  # merge (the default strategy)
hint:   git config pull.rebase true   # rebase
hint:   git config pull.ff only       # fast-forward only
hint: 
hint: You can replace "git config" with "git config --global" to set a default
hint: preference for all repositories. You can also pass --rebase, --no-rebase,
hint: or --ff-only on the command line to override the configured default per
hint: invocation.


Already up to date.


ERROR: Package 'trans-gemma' requires a different Python: 3.9.2 not in '>=3.10'


✅ trans-gemma installed


## Step 3: HuggingFace 認證

**🔐 設定 Token**

需要：
1. Token: https://huggingface.co/settings/tokens
2. 接受模型: https://huggingface.co/google/gemma-2-2b-it

In [10]:
from huggingface_hub import login
import os
from pathlib import Path

# 依環境選擇認證方式
env_file = Path('trans-gemma/.env')
token = None

# Method 1: .env file (GCP/Local)
if env_file.exists():
    with open(env_file, 'r') as f:
        for line in f:
            if line.strip().startswith('HF_TOKEN='):
                token = line.split('=', 1)[1].strip().strip('"').strip("'")
                print("✅ Loaded from .env")
                break

# Method 2: Colab Secrets
if not token and ENV == 'colab':
    try:
        from google.colab import userdata
        token = userdata.get('HF_TOKEN')
        print("✅ Loaded from Colab Secrets")
    except:
        pass

# Method 3: Environment variable
if not token:
    token = os.getenv('HF_TOKEN') or os.getenv('HUGGING_FACE_HUB_TOKEN')
    if token:
        print("✅ Loaded from environment")

# Method 4: Manual input
if not token:
    print("\n📝 Please enter HuggingFace Token:")
    if ENV == 'colab':
        print("   💡 Tip: Use Colab Secrets (🔑 icon) for better security")
    print("   1. Get token: https://huggingface.co/settings/tokens")
    print("   2. Accept model: https://huggingface.co/google/gemma-2-2b-it\n")
    
    token = input("Token: ").strip()
    
    # Save to .env (not for Colab)
    if ENV != 'colab':
        env_file.parent.mkdir(exist_ok=True)
        with open(env_file, 'w') as f:
            f.write(f"HF_TOKEN={token}\n")
        print("💾 Saved to .env")

# Login
os.environ['HF_TOKEN'] = token
login(token=token)
print("\n✅ HuggingFace authentication complete!")


📝 Please enter HuggingFace Token:
   1. Get token: https://huggingface.co/settings/tokens
   2. Accept model: https://huggingface.co/google/gemma-2-2b-it



Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


💾 Saved to .env

✅ HuggingFace authentication complete!


## Step 4: GPU 檢查

**確認 GPU 可用**

In [11]:
import torch
import subprocess

print("="*80)
print("🎮 GPU Check")
print("="*80)

# Check PyTorch CUDA
print(f"✅ PyTorch: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    
    # Check nvidia-smi
    try:
        result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'],
                              capture_output=True, text=True, timeout=3)
        if result.returncode == 0:
            print(f"\n📊 nvidia-smi: {result.stdout.strip()}")
    except:
        pass
else:
    print("\n⚠️  WARNING: No GPU detected!")
    print("   Translation will be very slow on CPU.")
    if ENV == 'colab':
        print("\n   🔧 Enable GPU in Colab:")
        print("      Runtime → Change runtime type → T4 GPU → Save")

print("="*80)

🎮 GPU Check
✅ PyTorch: 2.5.1+cu121
✅ CUDA available: True
✅ GPU: Tesla T4
✅ VRAM: 14.6 GB

📊 nvidia-smi: Tesla T4, 15360 MiB


## Step 5: 設定翻譯參數

**✏️ 修改這裡來翻譯不同論文**

In [12]:
# ============================================
# 翻譯設定（修改這裡）
# ============================================

# arXiv 論文 ID
ARXIV_ID = "2403.08295"  # Gemma paper

# 要翻譯的章節（頁碼範圍）
SECTIONS = {
    "abstract": (1, 1),  # 第 1 頁
}

# 語言
SOURCE_LANG = "en"
TARGET_LANG = "zh-TW"

# 選項
SAVE_HTML = True  # 儲存互動式 HTML

# ============================================
# 摘要
# ============================================

total_pages = sum(end - start + 1 for start, end in SECTIONS.values())

print("="*80)
print("🎯 Translation Configuration")
print("="*80)
print(f"📄 Paper: arXiv:{ARXIV_ID}")
print(f"🌐 {SOURCE_LANG} → {TARGET_LANG}")
print(f"📚 Sections: {list(SECTIONS.keys())}")
print(f"📊 Pages: {total_pages}")
print(f"⏱️  Time: ~{total_pages * 23}s")
print(f"💾 HTML: {'Yes' if SAVE_HTML else 'No'}")
print("="*80)

🎯 Translation Configuration
📄 Paper: arXiv:2403.08295
🌐 en → zh-TW
📚 Sections: ['abstract']
📊 Pages: 1
⏱️  Time: ~23s
💾 HTML: Yes


## Step 6: 載入模型

**🚀 載入 TranslateGemma**

⚠️ 首次執行會下載 ~8GB 模型

In [16]:
import sys
sys.path.insert(0, '/root/trans-gemma/examples')
sys.path.insert(0, '/root/trans-gemma/examples/backends')

from transformers_backend import TransformersBackend
import inspect

# 檢查 __init__ 的參數
print("TransformersBackend.__init__ 參數：")
print(inspect.signature(TransformersBackend.__init__))


TransformersBackend.__init__ 參數：
(self)


In [ ]:
import sys
import torch

# 動態設定 trans-gemma 路徑
if ENV == 'gcp':
    base_path = '/root/trans-gemma'
else:
    base_path = 'trans-gemma'

sys.path.insert(0, f'{base_path}/examples')
sys.path.insert(0, f'{base_path}/examples/backends')

from transformers_backend import TransformersBackend

print("🚀 Loading TranslateGemma (4B)...")
print("   ⏳ Downloading model (~8GB) on first run...\n")

# 1. 創建 backend 實例（預設就是 translategemma-4b-it）
backend = TransformersBackend()

# 2. 載入模型
result = backend.load_model()

print("\n✅ Model loaded!")
print(f"📍 Device: {result['metadata']['device']}")
print(f"📊 Load time: {result['load_time']:.1f}s")
print(f"💾 Memory: {result['metadata']['available_memory_gb']:.1f} GB available")
print("🎉 Ready to translate!\n")

## Step 7: 下載 PDF & 翻譯

**執行翻譯任務**

In [18]:
import urllib.request
import fitz
import time
import gc
from tqdm.auto import tqdm

# Download PDF
def download_arxiv(arxiv_id):
    clean_id = arxiv_id.split('v')[0] if 'v' in arxiv_id else arxiv_id
    url = f"https://arxiv.org/pdf/{clean_id}.pdf"
    filename = f"{arxiv_id}.pdf"
    
    print(f"📥 Downloading arXiv:{arxiv_id}")
    urllib.request.urlretrieve(url, filename)
    
    doc = fitz.open(filename)
    total = len(doc)
    doc.close()
    
    print(f"✅ Downloaded: {filename} ({total} pages)\n")
    return filename, total

def extract_text(pdf_path, page_num):
    doc = fitz.open(pdf_path)
    text = doc[page_num - 1].get_text()
    doc.close()
    return text.strip()

# Download
pdf_path, total_pdf_pages = download_arxiv(ARXIV_ID)

# Translate
results = []
total_to_translate = sum(end - start + 1 for start, end in SECTIONS.values())

print("="*80)
print("🚀 Translation Started")
print("="*80)
print(f"📊 Pages: {total_to_translate}")
print(f"⏱️  Time: ~{total_to_translate * 23}s\n")

with tqdm(total=total_to_translate, desc="📖 Translating", unit="page") as pbar:
    for section, (start, end) in SECTIONS.items():
        for page_num in range(start, end + 1):
            pbar.set_description(f"📖 {section.capitalize()} - Page {page_num}")
            
            text = extract_text(pdf_path, page_num)
            if not text or len(text) < 10:
                pbar.write(f"⚠️  Page {page_num}: No text")
                pbar.update(1)
                continue
            
            start_time = time.time()
            result = backend.translate(text, source_lang=SOURCE_LANG, target_lang=TARGET_LANG)
            elapsed = time.time() - start_time
            
            results.append({
                'page_num': page_num,
                'section': section,
                'original': text,
                'translation': result['translation'],
                'time': elapsed
            })
            
            pbar.write(f"✅ Page {page_num}: {elapsed:.2f}s")
            pbar.update(1)
            
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            gc.collect()

print("\n" + "="*80)
print("✅ Translation Complete!")
print("="*80)
print(f"📊 Pages: {len(results)}")
total_time = sum(r['time'] for r in results)
print(f"⏱️  Total: {total_time:.1f}s")
print(f"⚡ Avg: {total_time/len(results):.1f}s/page")
print("="*80)

📥 Downloading arXiv:2403.08295
✅ Downloaded: 2403.08295.pdf (17 pages)

🚀 Translation Started
📊 Pages: 1
⏱️  Time: ~23s



📖 Translating:   0%|          | 0/1 [00:00<?, ?page/s]

The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✅ Page 1: 187.38s

✅ Translation Complete!
📊 Pages: 1
⏱️  Total: 187.4s
⚡ Avg: 187.4s/page


## Step 8: 顯示結果

**查看翻譯**

In [19]:
for r in results:
    print("\n" + "="*80)
    print(f"📄 Page {r['page_num']} - {r['section'].upper()}")
    print("="*80)
    
    print("\n📝 Original:")
    print("-" * 80)
    print(r['original'][:300] + "..." if len(r['original']) > 300 else r['original'])
    
    print("\n🌐 Translation:")
    print("-" * 80)
    print(r['translation'][:300] + "..." if len(r['translation']) > 300 else r['translation'])
    
    print(f"\n⏱️  {r['time']:.2f}s")
    print("="*80)


📄 Page 1 - ABSTRACT

📝 Original:
--------------------------------------------------------------------------------
2024-02-21
Gemma: Open Models Based on Gemini
Research and Technology
Gemma Team, Google DeepMind1
This work introduces Gemma, a family of lightweight, state-of-the art open models built from the research
and technology used to create Gemini models. Gemma models demonstrate strong performance across...

🌐 Translation:
--------------------------------------------------------------------------------
論文摘要：
Gemma 是一系列基于 Gemini 的轻量级、先进的开源模型。这些模型在语言理解、推理和安全性等方面的表现优异。我们发布了两个不同大小的模型（70 亿和 20 亿参数），并提供了预训练和微调的检查点。Gemma 在 18 项基于文本的任务中，优于同等规模的开源模型，我们还提供了模型安全性和责任方面的全面评估，以及详细的模型开发描述。我们相信，负责任地发布大型语言模型对于提高前沿模型的安全性，以及促进下一代大型语言模型创新的关键。

⏱️  187.38s


## Step 9: 生成 HTML

**💾 儲存互動式網頁**

In [ ]:
if SAVE_HTML:
    from datetime import datetime
    import os
    
    # Generate HTML (simplified version)
    pages_html = ""
    for idx, r in enumerate(results):
        display_style = "" if idx == 0 else 'style="display: none;"'
        pages_html += f"""
        <div id="page-{idx}" class="page" {display_style}>
            <div class="header">
                <h2>📄 {r['section'].upper()} - Page {r['page_num']}</h2>
                <span>⏱️ {r['time']:.2f}s</span>
            </div>
            <div class="columns">
                <div class="col"><h3>Original</h3><pre>{r['original']}</pre></div>
                <div class="col"><h3>Translation</h3><pre>{r['translation']}</pre></div>
            </div>
        </div>
        """
    
    html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="UTF-8">
        <title>arXiv:{ARXIV_ID} - TranslateGemma</title>
        <style>
            body {{ font-family: sans-serif; max-width: 1400px; margin: 0 auto; padding: 20px; background: #f5f5f5; }}
            h1 {{ text-align: center; color: #667eea; }}
            .nav {{ position: sticky; top: 0; background: white; padding: 15px; box-shadow: 0 2px 8px rgba(0,0,0,0.1);
                    display: flex; justify-content: space-between; margin-bottom: 20px; border-radius: 8px; }}
            button {{ padding: 10px 20px; background: #667eea; color: white; border: none; border-radius: 4px; cursor: pointer; }}
            button:disabled {{ opacity: 0.5; }}
            .page {{ background: white; margin: 20px 0; border-radius: 8px; overflow: hidden; }}
            .header {{ background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 15px 20px;
                       display: flex; justify-content: space-between; }}
            .columns {{ display: grid; grid-template-columns: 1fr 1fr; gap: 0; }}
            .col {{ padding: 20px; }}
            .col:first-child {{ background: #f8f9fa; border-right: 2px solid #ddd; }}
            pre {{ white-space: pre-wrap; line-height: 1.8; margin: 0; }}
        </style>
        <script>
            let c=0,t={len(results)},p={[r['page_num'] for r in results]};
            function show(i){{if(i<0||i>=t)return;document.querySelectorAll('.page').forEach(p=>p.style.display='none');
            document.getElementById('page-'+i).style.display='block';c=i;update();window.scrollTo({{top:0}});}}
            function update(){{document.getElementById('prev').disabled=(c===0);
            document.getElementById('next').disabled=(c===t-1);
            document.getElementById('info').textContent=`Page ${{p[c]}} (${{c+1}}/${{t}})`;}}
            document.addEventListener('keydown',e=>{{if(e.key==='ArrowLeft')show(c-1);if(e.key==='ArrowRight')show(c+1);}});
            window.onload=update;
        </script>
    </head>
    <body>
        <h1>📄 arXiv:{ARXIV_ID} Bilingual Translation</h1>
        <p style="text-align: center; color: #666;">{SOURCE_LANG} → {TARGET_LANG} | {datetime.now().strftime("%Y-%m-%d %H:%M")}</p>
        <div class="nav">
            <button id="prev" onclick="show(c-1)">◀ Prev</button>
            <span id="info"></span>
            <button id="next" onclick="show(c+1)">Next ▶</button>
        </div>
        <p style="text-align: center; background: #fff3cd; padding: 10px; border-radius: 4px;">💡 Use ← → keys</p>
        {pages_html}
    </body>
    </html>
    """
    
    filename = f"arxiv_{ARXIV_ID}_{SOURCE_LANG}-{TARGET_LANG}.html"
    filepath = os.path.abspath(filename)
    
    # Save HTML file
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(html)
    
    print(f"💾 HTML saved: {filename}")
    print(f"📂 Full path: {filepath}")
    print(f"📊 Size: {os.path.getsize(filename) / 1024:.1f} KB")
    print(f"📄 Pages: {len(results)}")
    
    # Show preview of first translation
    print(f"\n🔍 Preview (first 200 chars):")
    print("-" * 80)
    first_translation = results[0]['translation'][:200]
    print(first_translation + "..." if len(results[0]['translation']) > 200 else first_translation)
    print("-" * 80)
    
    print(f"\n📥 To view the full HTML:")
    print(f"   1. Download: Right-click '{filename}' in Files panel → Download")
    print(f"   2. Or use path: {filepath}")
    
    # Try auto-download for native Colab (fails gracefully if remote)
    if ENV == 'colab':
        try:
            from google.colab import files
            files.download(filename)
            print(f"\n✅ Auto-download triggered (native Colab only)")
        except Exception as e:
            print(f"\n💡 For VSCode remote access:")
            print(f"   → Open Files panel (left sidebar)")
            print(f"   → Find '{filename}'")
            print(f"   → Right-click → Download")
else:
    print("⏭️  HTML generation skipped")

## 🎉 完成！

### 下一步

1. **翻譯其他論文**: 修改 Step 5 的 `ARXIV_ID`
2. **查看 HTML**: 在瀏覽器開啟下載的檔案
3. **分享**: 歡迎分享到社群媒體 🚀

### 相關連結

- [GitHub](https://github.com/jimmyliao/trans-gemma)
- [TranslateGemma Paper](https://arxiv.org/abs/2601.09012)
- [Gemma Model](https://huggingface.co/google/gemma-2-2b-it)

**Made with ❤️ by Jimmy Liao**